In [1]:
import json
import os
import numpy as np
import glob
import datetime
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import clear_output

In [2]:
def toTime(time):
    year, month, day = int(time[:4]), int(time[5:7]), int(time[8:10])
    hour, minute, second = int(time[11:13]), int(time[14:16]), int(time[17:19])

    if len(time) > 19:
        microsec = int(time[20:-1])
        return datetime.datetime(year, month, day, hour, minute, second, microsec)
    else:
        return datetime.datetime(year, month, day, hour, minute, second)

In [3]:
def fileTime(time):
    year, month, day = time[:4], time[5:7], time[8:10]
    hour, minute, second = time[11:13], time[14:16], time[17:19]
    
    return year+month+day+hour+minute+second

In [4]:
def genDuration(start, ptime, stime, sample_rate, length):
    
    # time: str -> datetime.datetime
    start, ptime, stime = toTime(start), toTime(ptime), toTime(stime)
    
    # ptime, stime 減去儀器開始時間，以得到具體在哪個 sample point
    diff_p, diff_s = ptime-start, stime-start
    sec_diff_p, microsec_diff_p = diff_p.seconds, diff_p.microseconds
    sec_diff_s, microsec_diff_s = diff_s.seconds, diff_s.microseconds
    
    p_trigger_sample = int((sec_diff_p+microsec_diff_p/1e+5)*sample_rate)
    s_trigger_sample = int((sec_diff_s+microsec_diff_s/1e+5)*sample_rate)

    # 生成 ground-truth
    gt = np.zeros(length)
    gt[p_trigger_sample:s_trigger_sample] = 1
    
    return gt

In [5]:
# 速度微分 -> 加速度
# 100.0: sampling_rate
def v_to_a(z, n, e, sample_rate):
    d_z = np.gradient(z, 1.0/sample_rate)
    d_n = np.gradient(n, 1.0/sample_rate)
    d_e = np.gradient(e, 1.0/sample_rate)

    return d_z, d_n, d_e

In [6]:
def check_csv_exist(ori_time, station, instrument, stage, root_save_path):
    ori_time = fileTime(ori_time)
    
    file_name = ori_time + '_' + station + '_' + instrument
    base_path = os.path.join(stage, file_name)
    save_path = os.path.join(root_save_path, base_path)
    
    if os.path.exists(save_path+'.csv'):
        return True
    else:
        return False

In [7]:
def toCSV(z, n, e, gt, ori_time, station, instrument, stage, root_save_path, cut):
    cut = int(cut)
    z, n, e, gt = z[:cut], n[:cut], e[:cut], gt[:cut]
    to_write = np.vstack((e, n, z, gt))
    ori_time = fileTime(ori_time)
   
    # filename: 地震發生時間 + 測站名 + 儀器種類
    file_name = ori_time + '_' + station + '_' + instrument
    base_path = os.path.join(stage, file_name)
    save_path = os.path.join(root_save_path, base_path)
    print(save_path)
    np.savetxt(save_path+'.csv', to_write, delimiter=',')

In [8]:
def check_wave(wave):
    if np.mean(wave) == 0.0 and np.var(wave) == 0.0:
        return False
    else:
        return True

### 取地震波形:
    
    1. 先檢查 s_arrival_time 是否有效(有些 pfile 中的值=0) && 篩選震央距離或震度來取 waveforms
        
    2. 取 Z, N, E, factor, p_arrival_time, s_arrival_time
    
    3. 確定資料類型是 Velocity or Acceleration，if Velocity: 微分成 acceleration
    
    4. Label 出 p-wave duration (ground-truth)
    
    5. 匯入 csv

In [25]:
def getSeisWave(p, root_save_path):
    for k in p.keys():
        try:
            # 看測站內有多少組波形資料
            n_data = p[k]['numberOfData']
            clear_output(wait=True)
            for w in range(n_data):
                
                # 若儀器失效或是沒有抓到 s波到時就不要存成訓練資料
                if (not p[k][str(w)]['DataAvailable']['instrument']) or (not p[k][str(w)]['DataAvailable']['Stime']):
                    continue
                
                # 只挑 instrument=FBA
                if p[k][str(w)]['instrument'] != 'FBA':
                    continue
                    
                # get Z, N, E, convert to ndarray
                z, n, e = p[k][str(w)]['Z'], p[k][str(w)]['N'], p[k][str(w)]['E']
                z, n, e = np.array(z), np.array(n), np.array(e)
                
                # multiply z, n, e by factor
                z, n, e = z*p[k][str(w)]['factor'][0], n*p[k][str(w)]['factor'][1], e*p[k][str(w)]['factor'][2]
                
                ori_time, instrument = p['ori_time'], p[k][str(w)]['instrument']
                
                # 檢查波形有效性
                if (not check_wave(z)) or (not check_wave(n)) or (not check_wave(e)) or \
                    (check_csv_exist(ori_time, k, instrument, 'seis', root_save_path)):
                    continue
                
                # get the multiple attributes
                waveType = p[k][str(w)]['datatype']
                sampleRate = p[k][str(w)]['sampling_rate']
                starttime = p[k][str(w)]['starttime']
                ptime, stime = p[k][str(w)]['p_arrival_time'], p[k][str(w)]['s_arrival_time']
                
                # if datatype='velocity', 積分成 acceleration
                if waveType == 'velocity':
                    z, n, e = v_to_a(z, n, e, sampleRate)
                    
                # label p-wave duration
                gt = genDuration(starttime, ptime, stime, sampleRate, z.shape[0])
                
                # Display info
                #print(f"event time: {p['ori_time']}, station: {k}")
                draw(z, n, e, gt)
                
                # Check manually
                print('p_weight, s_weight: (%.2f, %.2f)' %(p[k]['p_weight'], p[k]['s_weight']))
                available = input("Keep waveform? (0: Discard, 1: Keep it) ")
                if available == '1':
                    # 決定切在哪邊
                    cut = input("Where to cut? (sample) ")
                    
                    # write into csv file
                    toCSV(z, n, e, gt, ori_time, k, instrument, 'seis', root_save_path, cut)
                else:
                    continue
        except Exception as e:
            pass
#             print(e)

In [10]:
def draw(z, n, e, gt):
    plt.figure(figsize=(35, 25))
    plt.subplot(411)
    plt.title('Z')
    plt.plot(z)
    
    plt.subplot(412)
    plt.title('N')
    plt.plot(n)
    
    plt.subplot(413)
    plt.title('E')
    plt.plot(e)
    
    plt.subplot(414)
    plt.title('GT')
    plt.plot(gt)
    plt.show()

In [ ]:
# finish: 
root = "/mnt/nas6/CWBDatasets/2015/"
root_save_path = "/mnt/nas7/weiwei/earthquake_RNN/newTrain/"
files = os.listdir(root)

In [ ]:
for f in files:
    print(f)
    f = open(os.path.join(root, f), 'r')
    p = json.load(f)
    
    getSeisWave(p, root_save_path)